In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import itertools

from vivarium import Artifact, InteractiveContext
import ipywidgets
import pandas as pd, numpy as np, os
#import seaborn as sns
pd.set_option('display.max_rows', 8)

In [2]:
input_draws = [0]
random_seeds = [0]
x_factor_effects = [1.3]
mild_to_tmrel_transitions = [100,500,10]
data = pd.DataFrame()
for input_draw in input_draws:
    for random_seed in random_seeds:
        for x_factor_effect in x_factor_effects:
            for mild_to_tmrel_transition in mild_to_tmrel_transitions:
                data = data.append(pd.read_csv(f'draw_{input_draw}_seed_{random_seed}_x_factor_{x_factor_effect}_mild_transition_{mild_to_tmrel_transition}/data.csv'))
data = data.append(pd.read_csv(f'draw_0_seed_0_x_factor_2_mild_transition_100/data.csv'))
data = data.rename(columns={'Unnamed: 0':'simulant'})
data                

/ihme/code/alibow/miniconda/envs/vivarium_ciff_sam/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (16,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,simulant,tracked,years_of_life_lost,cause_of_death,sex,age,exit_time,entrance_time,alive,location,...,status,mild_child_wasting_event_time,timepoint,x_factor_exposure,wasting_treatment_exposure,population,input_draw,random_seed,x_factor_effect,mild_to_tmrel_transition
0,739,True,0.0,not_dead,Male,1.844927,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
1,4971,True,0.0,not_dead,Female,3.021960,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
2,5575,True,0.0,not_dead,Male,1.282061,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,defaulted_from_sam_to_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
3,5878,True,0.0,not_dead,Male,1.605293,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,defaulted_from_sam_to_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16732,9988,True,0.0,not_dead,Female,2.997209,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,NaN,NaN,endline,cat2,cat2,controls,0,0,2.0,100
16733,9990,True,0.0,not_dead,Female,2.491523,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,NaN,2021-12-31 12:00:00,endline,cat2,cat1,controls,0,0,2.0,100
16734,9991,True,0.0,not_dead,Male,3.292968,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,NaN,2022-08-26 00:00:00,endline,cat1,cat2,controls,0,0,2.0,100
16735,9994,True,0.0,not_dead,Female,4.551153,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,NaN,2023-01-15 12:00:00,endline,cat2,cat2,controls,0,0,2.0,100


In [3]:
cols = ['input_draw','x_factor_effect','mild_to_tmrel_transition','population','timepoint','simulant','status']

def calculate_episode_frequency(status, severity):
    baseline = (data.loc[data.status==status]
                    .set_index(['input_draw','x_factor_effect','mild_to_tmrel_transition','simulant'])
                    [['mild_child_wasting_event_count','susceptible_to_child_wasting_event_count',
                      'moderate_acute_malnutrition_event_count','severe_acute_malnutrition_event_count']])
    endline = (data.loc[(data.population=='cohort') & (data.timepoint=='endline')]
                   .set_index(['input_draw','x_factor_effect','mild_to_tmrel_transition','simulant'])
                  [['mild_child_wasting_event_count','susceptible_to_child_wasting_event_count',
                    
                      'moderate_acute_malnutrition_event_count','severe_acute_malnutrition_event_count',
                   'x_factor_exposure','wasting_treatment_exposure','sex','age','cause_of_death']])
    res = baseline.merge(endline, right_index=True, left_index=True, suffixes=['_baseline','_endline']).reset_index()
    for sev in ['mild_child_wasting','susceptible_to_child_wasting','severe_acute_malnutrition','moderate_acute_malnutrition']:
        res[f'{sev}_episodes'] = res[f'{sev}_event_count_endline'] - res[f'{sev}_event_count_baseline']
    res['count'] = 1
    res = (res.groupby(['input_draw','x_factor_effect','mild_to_tmrel_transition',
                                 severity]).sum()
              / res.groupby(['input_draw','x_factor_effect','mild_to_tmrel_transition']).sum())[['count']].reset_index()
    return res

In [4]:
m = calculate_episode_frequency('recovered_from_mam','moderate_acute_malnutrition_episodes')
m.loc[m.moderate_acute_malnutrition_episodes==0]

,input_draw,x_factor_effect,mild_to_tmrel_transition,moderate_acute_malnutrition_episodes,count
0,0,1.3,10,0,0.549943
6,0,1.3,100,0,0.439141
13,0,1.3,500,0,0.256936
20,0,2.0,100,0,0.424908


In [5]:
m = calculate_episode_frequency('recovered_from_sam','severe_acute_malnutrition_episodes')
m['relapse'] = 1 - m['count']
m.loc[m.severe_acute_malnutrition_episodes==0]

,input_draw,x_factor_effect,mild_to_tmrel_transition,severe_acute_malnutrition_episodes,count,relapse
0,0,1.3,10,0,0.879310,0.120690
2,0,1.3,100,0,0.929825,0.070175
5,0,1.3,500,0,0.800000,0.200000
8,0,2.0,100,0,0.870370,0.129630


In [6]:
m = calculate_episode_frequency('recovered_from_sam','moderate_acute_malnutrition_episodes')
m['relapse'] = 1 - m['count']
m.loc[m.moderate_acute_malnutrition_episodes==0]

,input_draw,x_factor_effect,mild_to_tmrel_transition,moderate_acute_malnutrition_episodes,count,relapse
0,0,1.3,10,0,0.500000,0.500000
4,0,1.3,100,0,0.508772,0.491228
9,0,1.3,500,0,0.292308,0.707692
14,0,2.0,100,0,0.388889,0.611111


In [7]:
data.head()

,simulant,tracked,years_of_life_lost,cause_of_death,sex,age,exit_time,entrance_time,alive,location,...,status,mild_child_wasting_event_time,timepoint,x_factor_exposure,wasting_treatment_exposure,population,input_draw,random_seed,x_factor_effect,mild_to_tmrel_transition
0,739,True,0.0,not_dead,Male,1.844927,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
1,4971,True,0.0,not_dead,Female,3.021960,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
2,5575,True,0.0,not_dead,Male,1.282061,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,defaulted_from_sam_to_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
3,5878,True,0.0,not_dead,Male,1.605293,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,defaulted_from_sam_to_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
4,7830,True,0.0,not_dead,Male,3.748473,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_mam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100


In [8]:
severity='severe_acute_malnutrition'

baseline = (data.loc[(data.population=='controls') & (data.timepoint=='baseline')]
                .set_index(['input_draw','x_factor_effect','mild_to_tmrel_transition','simulant'])
                [['mild_child_wasting_event_count','susceptible_to_child_wasting_event_count',
                  'moderate_acute_malnutrition_event_count','severe_acute_malnutrition_event_count']])
endline = (data.loc[(data.population=='controls') & (data.timepoint=='endline')]
               .set_index(['input_draw','x_factor_effect','mild_to_tmrel_transition','simulant'])
              [['mild_child_wasting_event_count','susceptible_to_child_wasting_event_count',
                  'moderate_acute_malnutrition_event_count','severe_acute_malnutrition_event_count',
               'x_factor_exposure','wasting_treatment_exposure','sex','age','cause_of_death']])
h = baseline.merge(endline, right_index=True, left_index=True, suffixes=['_baseline','_endline']).reset_index()
for sev in ['mild_child_wasting','susceptible_to_child_wasting','severe_acute_malnutrition','moderate_acute_malnutrition']:
    h[f'{sev}_episodes'] = h[f'{sev}_event_count_endline'] - h[f'{sev}_event_count_baseline']
h['count'] = 1
h = (h.groupby(['input_draw','x_factor_effect','mild_to_tmrel_transition',
                             f'{severity}_episodes']).sum()
          / h.groupby(['input_draw','x_factor_effect','mild_to_tmrel_transition']).sum())[['count']].reset_index()
h=h.loc[h[f'{severity}_episodes']==0]
h['sam_occurance'] = 1-h['count']



h

,input_draw,x_factor_effect,mild_to_tmrel_transition,severe_acute_malnutrition_episodes,count,sam_occurance
0,0,1.3,10,0,0.927996,0.072004
5,0,1.3,100,0,0.929529,0.070471
9,0,1.3,500,0,0.938959,0.061041
13,0,2.0,100,0,0.926737,0.073263


In [9]:
data.loc[(data.status=='recovered_from_sam')&(data.mild_to_tmrel_transition==100)]

,simulant,tracked,years_of_life_lost,cause_of_death,sex,age,exit_time,entrance_time,alive,location,...,status,mild_child_wasting_event_time,timepoint,x_factor_exposure,wasting_treatment_exposure,population,input_draw,random_seed,x_factor_effect,mild_to_tmrel_transition
9,6858,True,0.0,not_dead,Male,3.724381,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
12,9561,True,0.0,not_dead,Male,4.334099,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
16,3143,True,0.0,not_dead,Male,3.928606,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
93,2610,True,0.0,not_dead,Male,0.583964,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,1.3,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,9222,True,0.0,not_dead,Female,0.885410,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,2.0,100
1231,9627,True,0.0,not_dead,Female,4.890496,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,2.0,100
1233,39,True,0.0,not_dead,Male,4.020816,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,2.0,100
1237,6467,True,0.0,not_dead,Female,1.639067,NaN,2021-12-31 12:00:00,alive,Ethiopia,...,recovered_from_sam,NaN,baseline,NaN,NaN,cohort,0,0,2.0,100
